In [7]:
# Example 15-1. Demonstration of a file object as a context manager
with open('/home/kwol/git/kw/jupyter/Fluent Python/module/lotto.py') as fp: #
    src = fp.read(60) #

print("len(src) : ", len(src))
print("fp : ", fp)

print("fp.closed, fp.encoding : ", fp.closed, ",", fp.encoding)
fp.read(60)

len(src) :  60
fp :  <_io.TextIOWrapper name='/home/kwol/git/kw/jupyter/Fluent Python/module/lotto.py' mode='r' encoding='UTF-8'>
fp.closed, fp.encoding :  True , UTF-8


ValueError: I/O operation on closed file.

In [13]:
# Example 15-2. Test driving the LookingGlass context manager class
class LookingGlass:

    def __enter__(self):  # <1>
        import sys
        self.original_write = sys.stdout.write  # <2>
        sys.stdout.write = self.reverse_write  # <3>
        return 'JABBERWOCKY'  # <4>

    def reverse_write(self, text):  # <5>
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):  # <6>
        import sys  # <7>
        sys.stdout.write = self.original_write  # <8>
        if exc_type is ZeroDivisionError:  # <9>
            print('Please DO NOT divide by zero!')
            return True  # <10>

with LookingGlass() as what:
    print(' :txet desrever', 'Alice, Kitty and Snowdrop')
    print(' :tahw', what)
print()
print('what: ', what)
print('Back to normal.')
txt = 'Alice, Kitty and Snowdrop'
print("txt[::-1] : ", txt[::-1]) # !!! reversed text

reversed text:  pordwonS dna yttiK ,ecilA
what:  YKCOWREBBAJ

what:  JABBERWOCKY
Back to normal.
txt[::-1] :  pordwonS dna yttiK ,ecilA


In [45]:
# Example 15-4. Exercising LookingGlass without a with block
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

from mirror import LookingGlass
manager = LookingGlass()
print("manager : ", manager)
monster = manager.__enter__()
print(" : 'YKCOWREBBAJ' == retsnom",  monster == 'JABBERWOCKY')
print(" : reganam", manager)

manager :  <mirror.LookingGlass object at 0x7f5fac1dc7f0>
monster == 'JABBERWOCKY' :  eurT
manager :  >0f7cd1caf5f7x0 ta tcejbo ssalGgnikooL.rorrim<


In [46]:
# Example 15-4. Exercising LookingGlass without a with block
manager.__exit__(None, None, None)
print("monster: ", monster)

monster:  JABBERWOCKY


In [48]:
# KWO: contextlib.closing
from contextlib import contextmanager

@contextmanager
def closing(thing):
    try:
        yield thing
    finally:
        thing.close()
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('http://www.python.org')) as page:
    for line in page:
        print(line)
        break

b'<!doctype html>\n'


In [51]:
# KWO: contextlib.suppress
import os
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove('somefile.tmp')

with suppress(FileNotFoundError):
    os.remove('someotherfile.tmp')

In [58]:
# KWO: ContextDecorator
from contextlib import ContextDecorator

class mycontext(ContextDecorator):
    def __enter__(self):
        print('Starting')
        return self

    def __exit__(self, *exc):
        print('Finishing')
        return False

@mycontext()
def function():
    print('The bit in the middle')

function()
print()
with mycontext():
    print('The bit in the middle')

Starting
The bit in the middle
Finishing

Starting
The bit in the middle
Finishing


In [60]:
# Example 15-5. mirror_gen.py: a context manager implemented with a generator.
import contextlib
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write
    
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [61]:
# Example 15-7. mirror_gen_exc.py: generator-based context manager implementing exception handling
import contextlib
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

In [63]:
# Example 15-8. A context manager for rewriting files in place.

from contextlib import contextmanager
import io
import os


@contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
    """Allow for a file to be replaced with new content.

    yields a tuple of (readable, writable) file objects, where writable
    replaces readable.

    If an exception occurs, the old file is restored, removing the
    written data.

    mode should *not* use 'w', 'a' or '+'; only read-only-modes are supported.

    """

    # move existing file to backup, create new file with same permissions
    # borrowed extensively from the fileinput module
    if set(mode).intersection('wa+'):
        raise ValueError('Only read-only file modes can be used')

    backupfilename = filename + (backup_extension or os.extsep + 'bak')
    try:
        os.unlink(backupfilename)
    except os.error:
        pass
    os.rename(filename, backupfilename)
    readable = io.open(backupfilename, mode, buffering=buffering,
                       encoding=encoding, errors=errors, newline=newline)
    try:
        perm = os.fstat(readable.fileno()).st_mode
    except OSError:
        writable = open(filename, 'w' + mode.replace('r', ''),
                        buffering=buffering, encoding=encoding, errors=errors,
                        newline=newline)
    else:
        os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
        if hasattr(os, 'O_BINARY'):
            os_mode |= os.O_BINARY
        fd = os.open(filename, os_mode, perm)
        writable = io.open(fd, "w" + mode.replace('r', ''), buffering=buffering,
                           encoding=encoding, errors=errors, newline=newline)
        try:
            if hasattr(os, 'chmod'):
                os.chmod(filename, perm)
        except OSError:
            pass
    try:
        yield readable, writable
    except Exception:
        # move backup back
        try:
            os.unlink(filename)
        except os.error:
            pass
        os.rename(backupfilename, filename)
        raise
    finally:
        readable.close()
        writable.close()
        try:
            os.unlink(backupfilename)
        except os.error:
            pass

import csv
with inplace('files/_init_units.csv', 'r', newline='') as (infh, outfh):
    reader = csv.reader(infh)
    writer = csv.writer(outfh)
    for row in reader:
        row += ['new', 'columns']
        writer.writerow(row)